In [1]:
from Scripts.utils import *  
import pandas as pd

columns = ['Date','Open','High','Low','Close','Volume','Yield','PercentageVolume',
           'SMA6','EMA6','WMA6','HMA6','SMA20','EMA20','WMA20','HMA20','SMA50','EMA50','WMA50','HMA50',
           'SMA100','EMA100','WMA100','HMA100','MACD','CCI','Stochastic Oscillator','RSI','ROC','PPO',
           'KST','BOLU','BOLD','BOLM']
data = pd.read_csv('DataReady/EURUSD/EURUSD_D1.csv', names = columns, header = 0)

In [2]:
toRemove = ['Volume', 'Date','High','Low','Open','Close']
df = selectData(data,toRemove)

In [3]:
normDf = normalizeData(df)
images = generateImages(normDf)

In [4]:
close = np.array(data['Close'].reset_index(drop=True))[28:]
labels = np.zeros((len(close)-28,28), dtype=float)
print(labels.shape)
for i in range(len(close)-28):
    labels[i,:] = close[i:i+28]
labels.shape

(4159, 28)


(4159, 28)

In [5]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(images[:-28], labels, test_size = 0.2,shuffle = True, random_state = 42)
train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y, test_size = 0.1,shuffle = True, random_state = 42)

In [6]:
train_X = np.array([elem.reshape(28*28,1) for elem in train_X])
valid_X = np.array([elem.reshape(28*28,1) for elem in valid_X])
test_X = np.array([elem.reshape(28*28,1) for elem in test_X])

In [7]:
epochs_set = [30,70]
batches = [32]
learning_rates = [0.0005,0.001,0.01,0.1]
num_channels = [2,3,4,5,6]
nb_filters = [25,64,100]

In [8]:
combinations = []
for epochs in epochs_set:
    for batch_size in batches:
        for learning_rate in learning_rates:
            for num_channel in num_channels:
                for nb_filter in nb_filters:
                    combinations.append([epochs, batch_size,learning_rate, num_channel, nb_filter])

In [10]:
from Scripts.TCNConfidence import *
preds = runLoadedModel(test_X, test_Y)

MSE: 0.0007491109802087384
CORR: 0.0875913376739729


In [ ]:
from Scripts.TCNConfidence import *

mse_min = 0.0007076702716970458
corr_max = 0.0993168912282981

best_factor = np.sqrt(mse_min) * (1-corr_max)
for comb in combinations:
    best = False
    epochs, batch_size,learning_rate, num_channel, nb_filter = comb
    predictedPrices, mse, corr = run(train_X, valid_X, test_X, train_Y, valid_Y,test_Y,
                                    epochs, batch_size,learning_rate, num_channel, nb_filter, mse_min, corr_max)
    print("{},{}".format(mse,corr))
    if corr == corr and corr>0:
        factor = np.sqrt(mse)*(1-corr)
        if factor < best_factor:
            mse_min = mse
            corr_max = corr
            best_factor = factor
            best = True
            print("BEST")
    f = open('Results/resultsTCNConfidence.txt','a+')
    f.write("{},{},{},{},3,{},{},{}, {}\n".format(epochs,batch_size,num_channel,nb_filter,learning_rate,mse, corr, best))
    f.close()

In [9]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(images[:-28], labels, test_size = 0.2,shuffle = False)

In [10]:
train_X = np.array([elem.reshape(28*28,1) for elem in train_X])
test_X = np.array([elem.reshape(28*28,1) for elem in test_X])

In [11]:
from Scripts.TCNConfidence import *
predictedPrices = runLoadedModel(test_X, test_Y)

MSE: 0.0005810566368506182
CORR: 0.12371208836346065


In [12]:
preds = np.zeros((predictedPrices.shape[0], predictedPrices.shape[0]+27))
for i in range(predictedPrices.shape[0]):
    preds[i,i:28+i] = predictedPrices[i,:]
predsMean = preds.sum(axis=1)/(preds != 0).sum(axis=1)
predsSigma = np.array([np.std([preds[i,j] for j in range(preds.shape[1]) if preds[i,j]!=0]) for i in range(preds.shape[0])])

In [15]:
future_test = np.array([elem.reshape(28*28,1) for elem in images[-28:]])
predictedPrices = runLoadedModel(future_test)

In [19]:
predsFuture = np.zeros((predictedPrices.shape[0], predictedPrices.shape[0]+27))
for i in range(predictedPrices.shape[0]):
    predsFuture[i,i:28+i] = predictedPrices[i,:]
predsMeanFuture = predsFuture.sum(axis=1)/(predsFuture != 0).sum(axis=1)
predsSigmaFuture = np.array([np.std([predsFuture[i,j] for j in range(predsFuture.shape[1]) if predsFuture[i,j]!=0]) for i in range(predsFuture.shape[0])])

In [23]:
predsSigmaFuture.shape

(28,)

In [24]:
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=[30,10])
plt.grid()
plt.plot(range(len(predsMean)),predsMean, label = 'predictedTCN', color = 'darkorange', marker = '^')
plt.plot(range(len(test_Y[:,0])),test_Y[:,0], label = 'true', color = 'blue', marker = 'o')
plt.fill_between(range(len(predsMean)), 
                     predsMean + 2 * predsSigma, 
                     predsMean - 2 * predsSigma, 
                     alpha=0.5, label='Uncertainty')
plt.plot(range(len(predsMean), len(predsMean)+len(predsMeanFuture)),predsMeanFuture, label = 'Future', color = 'green', marker = '^')
plt.fill_between(range(len(predsMean), len(predsMean)+len(predsMeanFuture)), 
                     predsMeanFuture + 2 * predsSigmaFuture, 
                     predsMeanFuture - 2 * predsSigmaFuture, 
                     alpha=0.5, label='Uncertainty Future')
plt.legend()
plt.show()